In [1]:
# 导入所需模块
import arcpy

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 设置环境
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"

# 筛选POI

In [7]:
# 输入的POI要素类
input_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_poi"

In [8]:
# 创建一个新的要素图层
input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)

In [5]:
# 定义不同功能区对应的POI类型
typecode_ranges = {
    'R': [str(num) for num in range(120300, 120305)],   # 居住用地
    'A': [f"{num:06d}" for num in range(80000, 80007)] +  # 从 080000 到 080006
        [f"{num:06d}" for num in range(80100, 80120)] +  # 从 080100 到 080119
        [f"{num:06d}" for num in range(80200, 80203)] +  # 从 080200 到 080202
        [f"{num:06d}" for num in range(80300, 80309)] +  # 从 080300 到 080308
        [f"{num:06d}" for num in range(141100, 141206)],  # 从 141100 到 141205  # 公共管理与公共服务设施用地
    'B': [f"{num:06d}" for num in range(50100, 50124)] +  # 从 050100 到 050123
        [f"{num:06d}" for num in range(50200, 50218)] +  # 从 050200 到 050217
        [f"{num:06d}" for num in range(60100, 60104)] +  # 从 060100 到 060103
        [f"{num:06d}" for num in range(60400, 60410)] +  # 从 060400 到 060409
        [f"{num:06d}" for num in range(61000, 61002)] +  # 从 061000 到 061001
        [f"{num:06d}" for num in range(160400, 160409)] +  # 从 160400 到 160408
        [f"{num:06d}" for num in range(160500, 160502)],   # 从 160500 到 160501   # 商业服务业设施用地
    'M': ["120100"] +  # 单独的值
        ["120202"] +  # 单独的值
        ["170000"] +  # 单独的值
        [f"{num:06d}" for num in range(170100, 170101)] +  # 从 170100 到 170100
        [f"{num:06d}" for num in range(170200, 170209)] +  # 从 170200 到 170208
        [f"{num:06d}" for num in range(170300, 170301)] +  # 从 170300 到 170300
        [f"{num:06d}" for num in range(170400, 170409)],   # 从 170400 到 170408    # 工业用地
    'S': [f"{num:06d}" for num in range(150104, 150108)] +  # 从 150104 到 150107
        [f"{num:06d}" for num in range(150200, 150211)] +  # 从 150200 到 150210
        [f"{num:06d}" for num in range(150300, 150308)] +  # 从 150300 到 150307
        ["180000"] +                                    # 单独的值
        [f"{num:06d}" for num in range(180100, 180105)] +  # 从 180100 到 180104
        [f"{num:06d}" for num in range(180200, 180204)] +  # 从 180200 到 180203
        [f"{num:06d}" for num in range(180300, 180303)] +  # 从 180300 到 180302
        ["180400", "180500"],                              # 单独的值 # 交通与道路设施用地
    'G': ["110000"] +  # 单独的值
        [f"{num:06d}" for num in range(110100, 110107)] +  # 从 110100 到 110106
        [f"{num:06d}" for num in range(110200, 110211)]    # 从 110200 到 110210 # 绿地与广场用地
}


In [9]:
for category, codes in typecode_ranges.items():
    # 创建查询
    query = "typecode IN ('{}')".format("', '".join(codes))

    # 按类型代码筛选POI
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

    # 保存筛选结果为新的要素类
    output_feature_class = "POI_{}".format(category)
    arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

    print("分类保存完成：", output_feature_class)

    # 清除当前的选择
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

print("所有分类完成")

分类保存完成： POI_R
分类保存完成： POI_A
分类保存完成： POI_B
分类保存完成： POI_M
分类保存完成： POI_S
分类保存完成： POI_G
所有分类完成


# 权重赋值

In [10]:
# R类型的Shapefile路径
shapefile_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_R"

In [11]:
# 添加权重字段
arcpy.AddField_management(shapefile_R, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_R, ["Weight"]) as cursor:
    for row in cursor:
        row[0] = 0.01  # R类型的权重为0.01
        cursor.updateRow(row)

print("权重更新完成 for 类型 R")

权重更新完成 for 类型 R


In [12]:
# A类型的Shapefile路径
shapefile_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_A"

In [13]:
# 添加权重字段
arcpy.AddField_management(shapefile_A, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_A, ["typecode", "Weight"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('14'):
            row[1] = 0.6706
        elif typecode.startswith('08'):
            row[1] = 0.501
        elif typecode.startswith('09'):
            row[1] = 0.5069
        elif typecode.startswith('13'):
            row[1] = 0.355
        cursor.updateRow(row)

print("权重更新完成 for 类型 A")

权重更新完成 for 类型 A


In [14]:
# B类型的Shapefile路径
shapefile_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_B"

In [15]:
# 添加权重字段
arcpy.AddField_management(shapefile_B, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_B, ["typecode", "Weight"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('05') or typecode.startswith('10'):
            row[1] = 0.5562
        elif typecode.startswith('06'):
            row[1] = 0.8146
        elif typecode.startswith('16') or typecode.startswith('12'):
            row[1] = 0.3057
        cursor.updateRow(row)

print("权重更新完成 for 类型 B")

权重更新完成 for 类型 B


In [16]:
# M类型的Shapefile路径
shapefile_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_M"

In [17]:
# 添加权重字段
arcpy.AddField_management(shapefile_M, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_M, ["Weight"]) as cursor:
    for row in cursor:
        row[0] = 0.3057  # M类型的权重为0.3057
        cursor.updateRow(row)

print("权重更新完成 for 类型 M")

权重更新完成 for 类型 M


In [18]:
# S类型的Shapefile路径
shapefile_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_S"

In [19]:
# 添加权重字段
arcpy.AddField_management(shapefile_S, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_S, ["typecode", "Weight"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('15'):
            row[1] = 1.0
        elif typecode.startswith('18'):
            row[1] = 0.01
        cursor.updateRow(row)

print("权重更新完成 for 类型 S")

权重更新完成 for 类型 S


In [20]:
# G类型的Shapefile路径
shapefile_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_G"

In [21]:
# 添加权重字段
arcpy.AddField_management(shapefile_G, "Weight", "DOUBLE")

# 更新权重
with arcpy.da.UpdateCursor(shapefile_G, ["typecode", "Weight"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('1101'):
            row[1] = 0.6548
        elif typecode.startswith('1102'):
            row[1] = 0.8245
        cursor.updateRow(row)

print("权重更新完成 for 类型 G")

权重更新完成 for 类型 G


# 空间连接

In [3]:
# 街区图层路径
block_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\block23"

In [3]:
# 添加 BlockID 字段
arcpy.AddField_management(block_layer, "BlockID", "LONG")

# 为每个街区赋予独特编码
counter = 1
with arcpy.da.UpdateCursor(block_layer, ["BlockID"]) as cursor:
    for row in cursor:
        row[0] = counter
        counter += 1
        cursor.updateRow(row)

print("街区编码完成")

街区编码完成


In [5]:
# 对类型 R 的 POI 进行空间连接
poi_layer_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_R"  # 类型 R 的 POI 图层
output_layer_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiR"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_R, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_R, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 R")


空间连接完成 for 类型 R


In [8]:
# 对类型 A 的 POI 进行空间连接
poi_layer_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_A"  # 类型 A 的 POI 图层
output_layer_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiA"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_A, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_A, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 A")

空间连接完成 for 类型 A


In [10]:
# 对类型 B 的 POI 进行空间连接
poi_layer_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_B"  # 类型 B 的 POI 图层
output_layer_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiB"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_B, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_B, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 B")

空间连接完成 for 类型 B


In [11]:
# 对类型 M 的 POI 进行空间连接
poi_layer_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_M"  # 类型 M 的 POI 图层
output_layer_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiM"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_M, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_M, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 M")

空间连接完成 for 类型 M


In [12]:
# 对类型 S 的 POI 进行空间连接
poi_layer_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_S"  # 类型 S 的 POI 图层
output_layer_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiS"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_S, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_S, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 S")

空间连接完成 for 类型 S


In [13]:
# 对类型 G 的 POI 进行空间连接
poi_layer_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\POI_G"  # 类型 G 的 POI 图层
output_layer_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiG"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_G, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_G, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 G")

空间连接完成 for 类型 G


## 将路网空间内的点归类到距离最近的街区单元中

In [4]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiR"
poi_layer_null_blockid = "Null_BlockID_Layer"

In [15]:
arcpy.MakeFeatureLayer_management(poi_layer, poi_layer_null_blockid,'"Join_Count" = 0')

<Result 'Null_BlockID_Layer'>

In [5]:

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid, block_layer)

# Near Analysis 将添加两个字段：NEAR_FID 和 NEAR_DIST


<Result 'Null_BlockID_Layer'>

In [7]:
# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成")


更新 BlockID 完成


In [9]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_A= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiA"
poi_layer_null_blockid_A = "Null_BlockID_Layer_A"

arcpy.MakeFeatureLayer_management(poi_layer_A, poi_layer_null_blockid_A,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_A, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_A, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR A")

更新 BlockID 完成 FOR A


In [14]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_B= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiB"
poi_layer_null_blockid_B = "Null_BlockID_Layer_B"

arcpy.MakeFeatureLayer_management(poi_layer_B, poi_layer_null_blockid_B,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_B, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_B, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR B")

更新 BlockID 完成 FOR B


In [15]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_M= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiM"
poi_layer_null_blockid_M = "Null_BlockID_Layer_M"

arcpy.MakeFeatureLayer_management(poi_layer_M, poi_layer_null_blockid_M,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_M, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_M, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR M")

更新 BlockID 完成 FOR M


In [16]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_S= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiS"
poi_layer_null_blockid_S = "Null_BlockID_Layer_S"

arcpy.MakeFeatureLayer_management(poi_layer_S, poi_layer_null_blockid_S,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_S, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_S, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR S")

更新 BlockID 完成 FOR S


In [17]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_G= r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiG"
poi_layer_null_blockid_G = "Null_BlockID_Layer_G"

arcpy.MakeFeatureLayer_management(poi_layer_G, poi_layer_null_blockid_G,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_G, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_G, ["NEAR_FID", "BlockID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "BlockID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR G")

更新 BlockID 完成 FOR G


# 将所有的POI汇总为poi23

In [20]:
import arcpy

# POI 文件列表及其对应的类别
poi_files_and_types = {
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiR": "R",
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiA": "A",
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiB": "B",
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiM": "M",
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiS": "S",
    r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiG": "G"
    
}

for poi_file, poi_type in poi_files_and_types.items():
    # 添加新字段
    arcpy.AddField_management(poi_file, "Type", "TEXT")

    # 填充新字段的值
    with arcpy.da.UpdateCursor(poi_file, ["Type"]) as cursor:
        for row in cursor:
            row[0] = poi_type
            cursor.updateRow(row)

In [4]:
# 要合并的要素类列表
poi_files = [r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiR",
                   r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiA",
                   r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiB",
                   r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiM",
                   r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiS",
                   r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\join_poiG"]

In [5]:
# 合并后的要素类输出路径
output_merged_poi = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\poi23"

# 合并操作
arcpy.Merge_management(inputs=poi_files, output=output_merged_poi)

print("POI 文件合并完成")

POI 文件合并完成


In [6]:
# 列出要保留的字段
fields_to_keep = ["name", "typecode", "wgs84x", "wgs84y", "weight", "BlockID", "type"]

# 获取合并后要素类的所有字段
fields = arcpy.ListFields(output_merged_poi)

# 删除不在保留列表中的字段
fields_to_delete = [f.name for f in fields if f.name not in fields_to_keep and not f.required]

if fields_to_delete:
    arcpy.DeleteField_management(output_merged_poi, fields_to_delete)

print("不需要的字段已删除")

不需要的字段已删除


# 计算指标值

In [ ]:
# 合并后的 POI 要素类
merged_poi = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\poi23"

In [3]:
# 用于存储统计信息的表
temp_stats_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\temp_stats"

In [16]:
# 计算每个街区中每类 POI 的数量
arcpy.analysis.Statistics(merged_poi, temp_stats_table, [["type", "COUNT"]], ["BlockID", "type"])

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\temp_stats'>

In [4]:
# 用于存储每类 POI 总数的表
total_counts_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\total_counts"

In [17]:
# 计算每类 POI 的总数
arcpy.analysis.Statistics(merged_poi, total_counts_table, [["type", "COUNT"]], ["type"])

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\total_counts'>

## 为统计表添加所需字段

In [5]:
# 添加 Frequency_Density 字段
arcpy.AddField_management(temp_stats_table, "Frequency_Density", "DOUBLE")

# 添加 Type_Ratio 字段
arcpy.AddField_management(temp_stats_table, "Type_Ratio", "DOUBLE")

print("字段添加完成")

字段添加完成


## 计算频率密度

In [6]:
# 读取每类 POI 的总数
total_counts = {}
with arcpy.da.SearchCursor(total_counts_table, ["type", "FREQUENCY"]) as cursor:
    for row in cursor:
        total_counts[row[0]] = row[1]

# 更新每个街区中每类 POI 的频率密度
with arcpy.da.UpdateCursor(temp_stats_table, ["type", "FREQUENCY", "Frequency_Density"]) as cursor:
    for row in cursor:
        type = row[0]
        count = row[1]
        # 计算频率密度
        row[2] = count / total_counts[type] if type in total_counts else 0
        cursor.updateRow(row)


## 计算类型比例

In [7]:
# 计算每个街区的总频率密度
block_total_density = {}
with arcpy.da.SearchCursor(temp_stats_table, ["BlockID", "Frequency_Density"]) as cursor:
    for row in cursor:
        block_id = row[0]
        density = row[1]
        if block_id not in block_total_density:
            block_total_density[block_id] = 0
        block_total_density[block_id] += density

# 计算类型比例
with arcpy.da.UpdateCursor(temp_stats_table, ["BlockID", "type", "Frequency_Density", "Type_Ratio"]) as cursor:
    for row in cursor:
        block_id = row[0]
        density = row[2]
        # 计算类型比例
        row[3] = density / block_total_density[block_id] if block_id in block_total_density else 0
        cursor.updateRow(row)


# tf-idf

In [9]:
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"

In [ ]:
poi_layer = "POI_2023.shp" # POI图层
land_use_layer = "LUCC2023.shp" # 用地功能图层
output_layer = "SpatialJoinOutput.shp" # 输出图层

In [ ]:
# 空间连接
arcpy.analysis.SpatialJoin(target_features = poi_layer,
                          join_features = land_use_layer,
                          out_feature_class = output_layer,
                          join_operation = "JOIN_ONE_TO_ONE",
                          join_type = "KEEP_ALL",
                          match_option = "INTERSECT")

In [ ]:
# 将空间连接的结果转换为DataFrame
fields = ['POI_Type', 'LandUse_Type', 'Shape_Area']
data_frame = pd.DataFrame.spatial.from_featureclass(output_layer)[fields]

# 计算频数
frequency = data_frame.groupby(['POI_Type', 'LandUse_Type']).size().reset_index(name = 'Frequency')

# 计算密度
density = data_frame.groupby(['POI_Type', 'LandUse_Type']).sum().reset_index()
density['Density'] = density['Frequency'] / density['Shape_Area']

# 计算类型比例
type_count = data_frame.gourpby('LandUse_Type').size().reset_index(name = 'Total_Count')
type_ratio = frequency.merge(type_count, on = 'LandUse_Type')
type_ratio['Type_Ratio'] = type_ratio['Frequency'] / type_ratio['Total_Count']

In [ ]:
# 核密度估计
kernel_density_output = "KernelDensityOutput"
arcpy.sa.KernelDensity(output_layer, "Count", kernel_density_output)

In [ ]:
plt.bar(type_ratio['POI_Type'],
       type_ratio['Type_Ratio'])
plt.xlabel('POI Type')
plt.ylabel('Type Ratio')
plt.title('POI Type Ratio in Different Land Use Areas')
plt.xticks(rotation = 45)
plt.show()

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 转换数据类型，每个城市功能区为一个“文档”，POI类型为“词汇”
data_frame['POI_Type'] = data_frame['POI_Type'].astype(str)
documents = data_frame.groupby('LandUse_Type')['POI_Type'].apply(lambda x: ''.join(x))

# 计算TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# 将TF-IDF结果转换为DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf_vectorizer.get_feature_names(), index = documents.index)